In [24]:
from matplotlib import pyplot as plt
import pandas as pd
from scipy.integrate import odeint
from astropy import cosmology
from IPython.display import Image
import numpy as np
%matplotlib inline
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 12

C =cosmology.Planck15

Describe migration between classes using differential equations parameterised by the transition rates.

In the interval $dt$, the **change** in the number of blue disks equals **minus** the fraction of the current number of blue disks that will become red disks **minus** the fraction of the current number of blue disks that will become red ellipticals **minus** term accounting for the number of blue disks moving into this mass bin due to increasing their mass via SF (Peng et al. 2010)

$\frac{dN_{BD}}{dt} = - N_{BD} \, \left(r_{BD \rightarrow RD} + r_{BD \rightarrow RE} +  (\alpha \cdot sSFR(t)) \right)$

In the interval $dt$, the **change** in the number of red disks equals the fraction of the current number of blue disks that will become red disks **minus** the fraction of the current number of red disks that will become red ellipticals (red disks do not rejuvenate in this model).

$\frac{dN_{RD}}{dt} = - N_{RD} \, r_{RD \rightarrow RE} + N_{BD} \, r_{BD \rightarrow RD}$


In the interval $dt$, the **change** in the number of red disks equals the fraction of the current number of blue disks that will become red ellipticals **minus** the fraction of the current number of red disks that will become red ellipticals (red ellipticals remain red ellipticals forever).

$\frac{dN_{RE}}{dt} = + N_{BD} \, r_{BD \rightarrow RE} + N_{RD} \, r_{RD \rightarrow RE}$

In [2]:
states = pd.Index(('BD', 'RD', 'RE')) #only considering blue disks, red disks, red ellipticals
rate_labels = pd.DataFrame(index='to ' + states, columns='from ' + states,
                     data=[['$r_{BD \rightarrow BD}$', '$r_{RD \rightarrow BD}$', '$r_{RE \rightarrow BD}$'],
                           ['$r_{BD \rightarrow RD}$', '$r_{RD \rightarrow RD}$', '$r_{RE \rightarrow RD}$'],
                           ['$r_{BD \rightarrow RE}$', '$r_{RD \rightarrow RE}$', '$r_{RE \rightarrow RE}$']])
rate_labels

,from BD,from RD,from RE
to BD,$r_{BD \rightarrow BD}$,$r_{RD \rightarrow BD}$,$r_{RE \rightarrow BD}$
to RD,$r_{BD \rightarrow RD}$,$r_{RD \rightarrow RD}$,$r_{RE \rightarrow RD}$
to RE,$r_{BD \rightarrow RE}$,$r_{RD \rightarrow RE}$,$r_{RE \rightarrow RE}$


In [3]:
def alpha(a_s,M_star,m):
    alpha = (1+a_s) - m/M_star
    return alpha
def sSFR(t):
    sSFR = 2.5*(t/3.5)**(-2.2)
    return sSFR
def rate_matrix(rates,m,t):
    #create matrix using rates to reproduce differential equations when dotted with vector N0
    rate_mtrx = pd.DataFrame(data=np.zeros((3, 3)))
    a_s = -1.4 
    M_star = 10.82
    rate_mtrx[0][0] = -rates['from BD']['to RD'] - rates['from BD']['to RE'] - alpha(a_s,M_star,m)*sSFR(t)
    rate_mtrx[0][1] = rates['from BD']['to RD']
    rate_mtrx[0][2] = rates['from BD']['to RE']
    rate_mtrx[1][2] = rates['from RD']['to RE']
    rate_mtrx[2][2] = rates['from RE']['to RE']
    return rate_mtrx
def model(N, t, rates, rate_matrix_fn, m):
    rate_mtrx = rate_matrix_fn(rates,m,t)
    return np.dot(rate_mtrx, N)
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]
def find_nearest_idx(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx


In [4]:
#centers of bins where f_rid and f_dir are evaluated 
z_centers = [0.3125, 0.53750000000000009, 0.76250000000000007, 0.98750000000000004]
z_centers = [round(z,2) for z in z_centers]
z_centers

[0.31, 0.54, 0.76, 0.99]

In [5]:
#High Mass: 11<M<11.4, measured numbers of each state at z = 0.3, 
rd = [114.1,99.3,34.9,21.1] 
bd = [266.2,253.7,119.3,72.8]
re = [310.9,365.7,124.1,104.9]

gzh_data = pd.DataFrame(index=states, columns=z_centers[::-1],
                        data=[bd,rd,re])
gzh_fracs = pd.DataFrame(index=['$f_{R|D}$','$f_{D|R}$'], columns = z_centers[::-1],
                        data = [[a/(a+b) for a,b in zip(rd,bd)],[a/(a+b) for a,b in zip(rd,re)]])
gzh_data=gzh_data.append(gzh_fracs)
gzh_data

,0.99,0.76,0.54,0.31
BD,266.200000,253.700000,119.300000,72.800000
RD,114.100000,99.300000,34.900000,21.100000
RE,310.900000,365.700000,124.100000,104.900000
$f_{R|D}$,0.300026,0.281303,0.226329,0.224707
$f_{D|R}$,0.268471,0.213548,0.219497,0.167460


In [6]:
zstart = 1.0
zend = 0.3
z = np.linspace(zstart, zend, 100)
t = C.age(z)
t0 = C.age(zend).value


In [7]:
def compute_likelihood(rate_vector,m):
    # Set rates per galaxy in Gyr^{-1} at fiducial redshift
    rates = pd.DataFrame(index='to ' + states, columns='from ' + states,
                     data=np.zeros((3, 3)))
    rates['from BD']['to RD'] = rate_vector[0]
    rates['from BD']['to RE'] = rate_vector[1]
    rates['from RD']['to RE'] = rate_vector[2]

    #set initial values
    N0 = pd.Series(index=states, data=[gzh_data[z_centers[-1]]['BD'], gzh_data[z_centers[-1]]['RD'], gzh_data[z_centers[-1]]['RE']])
    N0 /= N0.sum()  # normalise to unity
    
    #evolve numbers from z=1 to z=0.3
    N = odeint(model, N0, t, args=(rates, rate_matrix, m))
    N = pd.DataFrame(index=z, columns='$N_{' + states + '}$', data=N)
    N = N[::-1] #number evolution of BD,RD,RE
    F = pd.DataFrame(index=N.index) #evolution of fractions
    F['$f_{R|D}$'] = N['$N_{RD}$'] / (N['$N_{RD}$'] + N['$N_{BD}$'])
    F['$f_{D|R}$'] = N['$N_{RD}$'] / (N['$N_{RD}$'] + N['$N_{RE}$'])
    
    #compare results of fractions at 3 redshifts, compute chi-squared:
    chi2 = 0
    
    for zc in z_centers[:-1]: 
        obs_f_rid = gzh_data[zc]['$f_{R|D}$']
        obs_f_dir = gzh_data[zc]['$f_{D|R}$']
        idx = find_nearest(z,zc)
        model_f_rid = F['$f_{R|D}$'][idx]
        model_f_dir = F['$f_{D|R}$'][idx]
    
        chi2 += (obs_f_rid - model_f_rid)**2/model_f_rid
        chi2 += (obs_f_dir - model_f_dir)**2/model_f_dir

    #all done! return chi square value for these rates
    return chi2

Create a list of rates [bd->rd, bd->re, rd->re]:

In [101]:
n = 20.
rate_vectors = [[x/n,y/n,k/n] for x in range(1,int(n)) for y in range(1,int(n)) for k in range(1,int(n)) if (x/n+y/n)<=1]
len(rate_vectors)
#chi2=[compute_likelihood(rv,11) for rv in rate_vectors]


3610

Compute chi2 for each set of rates, compute timestamp after every 1000 combinations

In [102]:
chi2=[]
for i,rv in enumerate(rate_vectors):
    chi2.append(compute_likelihood(rv,11))
    if i % 1000 == 0:
        print('{}, {:%Y-%m-%d %H:%M:%S}'.format(i,datetime.datetime.now()))


0, 2017-05-18 13:01:58
1000, 2017-05-18 13:02:48
2000, 2017-05-18 13:03:41
3000, 2017-05-18 13:04:34


In [103]:
#sort chi2
chi2_sorted = [c for c in chi2]
chi2_sorted.sort()

In [108]:
#find the 10 lowest chi2 values 
for c in chi2_sorted[0:10]:
    idx = find_nearest_idx(chi2,c)
    print rate_vectors[idx],chi2[idx]

[0.05, 0.35, 0.05] 0.0305113028666
[0.1, 0.2, 0.95] 0.0314304590752
[0.05, 0.35, 0.1] 0.0319100715529
[0.1, 0.2, 0.9] 0.0319515917479
[0.1, 0.2, 0.85] 0.0328389880711
[0.1, 0.15, 0.95] 0.0330753642384
[0.05, 0.35, 0.15] 0.0336770325768
[0.1, 0.2, 0.8] 0.0341144683088
[0.1, 0.15, 0.9] 0.0351209677937
[0.05, 0.3, 0.05] 0.0357379670313
